In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
from asr_trainer import ASRTrainer, CVDataset, CTCLoss

In [3]:
asrt = ASRTrainer()
asrt.load_model()

cvd = CVDataset(asrt.processor)
ctc_loss = CTCLoss()

Loading model 'facebook/wav2vec2-base-960h' to device 'cpu'


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
from pprint import pprint
test_item = next(cvd.single_dataloader('test'))
pprint(test_item)

{'input_values': tensor([[1.9897e-05, 1.9897e-05, 1.9897e-05,  ..., 2.6142e-05, 2.6927e-05,
         3.3459e-05]]),
 'sentence': 'Joe Keaton disapproved of films, and Buster also had '
             'reservations about the medium.',
 'wav': tensor([[-7.3703e-13, -7.9181e-12, -7.6020e-12,  ...,  7.5047e-07,
          8.4476e-07,  1.6297e-06]])}


In [5]:
input_values = test_item["input_values"]
print(input_values.shape)
pred_sentence = asrt.predict_argmax(input_values)
print(pred_sentence)

torch.Size([1, 100800])
JO KEEPSAN DISAPPROVED OF THONES AND BUSTER ALSO HAD HESERVATIONS ABOUT THE MEDIUM


In [6]:
logits = asrt.get_logits(input_values)
print(logits.shape)
pred_ids = asrt.pred_ids_from_logits(logits)
print(pred_ids.shape)
target = ctc_loss.sentence_to_target(pred_sentence)
print(target["target"].shape)

print(pred_ids)
print(target["target"])

target_iter = iter(target["target"][0])
last = 0
for x in pred_ids[0]:
    if x != 0 and x != last:
        target_val = next(target_iter)
        if target_val == 0 and x == 4:
            pass #end of sentence
        else:
            assert x == target_val, f"No match for '{x}' - '{target_val}"
    last = x

torch.Size([1, 314, 32])
torch.Size([1, 314])
torch.Size([1, 300])
tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0, 29,  0,  0,  0,  0,  8,  0,  0,  0,  0,  0,  0,  0,  4,  0, 26,
          0,  0,  5,  0,  0,  5,  0, 23,  0,  0, 12,  0,  0,  0,  7,  0,  9,  0,
          0,  4,  4, 14,  0,  0, 10,  0, 12,  0,  0,  0,  0,  7, 23,  0,  0,  0,
         23,  0, 13,  0,  0,  0,  8,  8,  0, 25,  0,  0,  5, 14,  0,  4,  4,  0,
          0,  8, 20,  0,  4,  4,  4,  0,  0,  0,  6, 11,  0,  0,  0,  0,  8,  0,
          0,  0,  9,  5,  0,  0, 12, 12,  4,  4,  0,  0,  0,  0,  7,  9,  0, 14,
          0,  4,  4,  4,  0, 24,  0,  0,  0, 16,  0,  0, 12, 12,  0,  6,  0,  0,
          0,  5, 13, 13,  0,  4,  4,  0,  0,  7, 15,  0,  0,  0,  0, 12,  0,  0,
          8,  0,  4,  4,  0, 11,  0,  0,  7,  0,  0, 14,  4,  4,  4, 11,  0,  0,
          5,  0,  0,  0, 12,  0,  0,  5, 1

In [7]:
# target["target"][0] = 29
print(ctc_loss.singleLoss(logits, target))
target["target"][0] = 1
print(ctc_loss.singleLoss(logits, target))

tensor(0.3380)
tensor(31.7048)
